In [10]:
#!pip install opencv-python scikit-image ezdxf svgwrite

In [11]:
from skimage import io, color, measure
import svgwrite

def png_to_svg(png_path, svg_path):
    # Load the image
    image = io.imread(png_path)

    # Check if the image has an alpha channel (4 channels)
    if image.shape[-1] == 4:
        # Convert RGBA to RGB by removing the alpha channel
        image = color.rgba2rgb(image)

    # Convert to grayscale
    gray_image = color.rgb2gray(image)
    
    # Find contours
    contours = measure.find_contours(gray_image, 0.8)
    
    # Create an SVG file
    dwg = svgwrite.Drawing(svg_path, profile='tiny')
    
    # Add each contour to the SVG
    for contour in contours:
        # Scale the contour to fit the image
        contour = [(x, y) for x, y in contour]
        dwg.add(dwg.polyline(contour, stroke='black', fill='none'))
    
    # Save the SVG file
    dwg.save()

In [12]:
import ezdxf

def svg_to_dxf(svg_path, dxf_path):
    import xml.etree.ElementTree as ET
    
    doc = ezdxf.new('R2010')
    msp = doc.modelspace()

    tree = ET.parse(svg_path)
    root = tree.getroot()

    # Iterate over the paths in the SVG and add them to the DXF file
    for polyline in root.findall('.//{http://www.w3.org/2000/svg}polyline'):
        points = polyline.get('points').strip().split(' ')
        points = [tuple(map(float, p.split(','))) for p in points]
        msp.add_polyline2d(points)
    
    doc.saveas(dxf_path)



In [13]:
file = "tractor2"

png_to_svg(f'{file}.png', f'{file}.svg')
svg_to_dxf(f'{file}.svg', f'{file}.dxf')

PermissionError: [Errno 13] Permission denied: 'tractor2.dxf'